In [1]:
import pandas as pd
from sqlalchemy import create_engine
from datetime import date, timedelta
from pandas.tseries.offsets import BDay

engine = create_engine("mysql+pymysql://root:@localhost:3306/stock")
const = engine.connect()
engine = create_engine("sqlite:///c:\\ruby\\portlt\\db\\development.sqlite3")
conlt = engine.connect()

today = date.today()

### Set yesterday = last closed business day

In [2]:
num_business_days = BDay(1)
yesterday = today - num_business_days
yesterday = yesterday.date()

### Restart and Run All Cells

In [3]:
sqlUpd = """
UPDATE buy B
SET dividend =
(SELECT DIVIDEND FROM dividend D
WHERE B.name = D.name)"""
rp = const.execute(sqlUpd)

In [4]:
colu = 'name shares u_cost cost_amt price mkt_amt dividend div_amt cst-% mkt-% eps dpr-%'.split()
format_dict = {
    'shares':'{:,}', 
    'u_cost':'฿{:.2f}','cost_amt':'฿{:,.2f}',
    'price':'฿{:.2f}','mkt_amt':'฿{:,.2f}',
    'dividend':'฿{:.4f}','div_amt':'฿{:,.2f}','eps':'฿{:.4f}',    
    'cst-%':'{:,.2f}%','mkt-%':'{:,.2f}%','dpr-%':'{:,.2f}%'
}

### Rank

In [5]:
sql = '''
SELECT B.name, volbuy, B.price AS u_cost, 
dividend, P.price AS price, period AS prd
FROM buy B 
JOIN price P
ON B.name = P.name
WHERE P.date = "%s"
AND active = 1
ORDER BY period, name'''
sql = sql % today

In [6]:
df1 = pd.read_sql(sql, const)
df1.rename(columns={'volbuy': 'shares'}, inplace=True)
df1['shares'] = df1.shares.astype(int)
df1['cost_amt'] = round(df1['shares'] * df1['u_cost'], 2)
df1['mkt_amt'] = round(df1['shares'] * df1['price'], 2)
df1['div_amt'] = round(df1['shares'] * df1['dividend'], 2)
df1['mkt-%'] = round(df1['div_amt'] / df1['mkt_amt'] * 100,2)
df1['cst-%'] = round(df1['div_amt'] / df1['cost_amt'] * 100,2)

In [7]:
sql = '''
SELECT name, aq_eps AS eps 
FROM epss 
WHERE year = 2022 AND quarter = 4
'''
df2 = pd.read_sql(sql, conlt)

In [8]:
df_merge = pd.merge(df1, df2, on='name', how='inner')
df_merge['dpr-%'] = round(df_merge['dividend'] / df_merge['eps'] * 100,2)
df_merge.set_index('prd',inplace=True)
df_merge[colu].sort_values(['prd','cst-%'],ascending=[True,True]).style.format(format_dict)

,name,shares,u_cost,cost_amt,price,mkt_amt,dividend,div_amt,cst-%,mkt-%,eps,dpr-%
prd,,,,,,,,,,,,


In [9]:
import pandas as pd
from sqlalchemy import create_engine
from datetime import date, timedelta
from pandas.tseries.offsets import BDay

engine = create_engine("mysql+pymysql://root:@localhost:3306/stock")
const = engine.connect()
engine = create_engine("sqlite:///c:\\ruby\\portlt\\db\\development.sqlite3")
conlt = engine.connect()

today = date.today()

num_business_days = BDay(1)
yesterday = today - num_business_days
yesterday = yesterday.date()

sqlUpd = """
UPDATE buy B
SET dividend =
(SELECT DIVIDEND FROM dividend D
WHERE B.name = D.name)"""
rp = const.execute(sqlUpd)

colu = 'name shares u_cost cost_amt price mkt_amt dividend div_amt cst-% mkt-% eps dpr-%'.split()
format_dict = {
    'shares':'{:,}', 
    'u_cost':'฿{:.2f}','cost_amt':'฿{:,.2f}',
    'price':'฿{:.2f}','mkt_amt':'฿{:,.2f}',
    'dividend':'฿{:.4f}','div_amt':'฿{:,.2f}','eps':'฿{:.4f}',    
    'cst-%':'{:,.2f}%','mkt-%':'{:,.2f}%','dpr-%':'{:,.2f}%'
}

sql = '''
SELECT B.name, volbuy, B.price AS u_cost, 
dividend, P.price AS price, period AS prd
FROM buy B 
JOIN price P
ON B.name = P.name
WHERE P.date = "%s"
AND active = 1
ORDER BY period, name'''
sql = sql % yesterday

df1 = pd.read_sql(sql, const)
df1.rename(columns={'volbuy': 'shares'}, inplace=True)
df1['shares'] = df1.shares.astype(int)
df1['cost_amt'] = round(df1['shares'] * df1['u_cost'], 2)
df1['mkt_amt'] = round(df1['shares'] * df1['price'], 2)
df1['div_amt'] = round(df1['shares'] * df1['dividend'], 2)
df1['mkt-%'] = round(df1['div_amt'] / df1['mkt_amt'] * 100,2)
df1['cst-%'] = round(df1['div_amt'] / df1['cost_amt'] * 100,2)

sql = '''
SELECT name, aq_eps AS eps 
FROM epss 
WHERE year = 2022 AND quarter = 4
'''
df2 = pd.read_sql(sql, conlt)

df_merge = pd.merge(df1, df2, on='name', how='inner')
df_merge['dpr-%'] = round(df_merge['dividend'] / df_merge['eps'] * 100,2)
df_merge.set_index('prd',inplace=True)
df_merge[colu].sort_values(['prd','dpr-%'],ascending=[True,True]).style.format(format_dict)

,name,shares,u_cost,cost_amt,price,mkt_amt,dividend,div_amt,cst-%,mkt-%,eps,dpr-%
prd,,,,,,,,,,,,
1,PTTGC,"6,000",฿64.75,"฿388,500.00",฿45.75,"฿274,500.00",฿1.0000,"฿6,000.00",1.54%,2.19%,฿-1.9400,-51.55%
1,MCS,"75,000",฿15.40,"฿1,155,000.00",฿9.35,"฿701,250.00",฿0.2000,"฿15,000.00",1.30%,2.14%,฿0.2500,80.00%
1,TMT,"36,000",฿10.20,"฿367,200.00",฿7.85,"฿282,600.00",฿0.3000,"฿10,800.00",2.94%,3.82%,฿0.2700,111.11%
1,JASIF,"130,000",฿10.00,"฿1,300,000.00",฿7.10,"฿923,000.00",฿0.9200,"฿119,600.00",9.20%,12.96%,฿0.4563,201.62%
2,BANPU,"30,000",฿11.50,"฿345,000.00",฿10.80,"฿324,000.00",฿1.2000,"฿36,000.00",10.43%,11.11%,฿5.6490,21.24%
2,RCL,"27,000",฿38.75,"฿1,046,250.00",฿26.50,"฿715,500.00",฿7.0000,"฿189,000.00",18.06%,26.42%,฿29.7140,23.56%
2,SENA,"105,000",฿4.48,"฿470,400.00",฿3.50,"฿367,500.00",฿0.2053,"฿21,556.50",4.58%,5.87%,฿0.6271,32.74%
2,NER,"27,000",฿7.45,"฿201,150.00",฿5.95,"฿160,650.00",฿0.3800,"฿10,260.00",5.10%,6.39%,฿0.9640,39.42%
2,ORI,"45,000",฿11.00,"฿495,000.00",฿11.60,"฿522,000.00",฿0.7200,"฿32,400.00",6.55%,6.21%,฿1.5382,46.81%
